In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
np.random.seed(7)
from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.preprocessing import text

In [ ]:

train = pd.read_csv('/content/drive/My Drive/iQube/train_lstm2.csv')
test = pd.read_csv('/content/drive/My Drive/iQube/test_lstm2.csv')

In [ ]:
train.head(2)

,text,labels
0,and and,neutral
1,advice talk to your neighbours family to excha...,positive


In [ ]:
test.head(2)

,text,labels
0,trending new yorkers encounter empty supermark...,negative
1,when i couldnt find hand sanitizer at fred mey...,positive


In [ ]:
def target(label):
    if label == 'neutral': 
        return 0
    if label == 'positive':
        return 1
    else:
        return -1

In [ ]:
train['labels'] = train['labels'].apply(target)
train.head()

,text,labels
0,and and,0
1,advice talk to your neighbours family to excha...,1
2,coronavirus australia woolworths to give elder...,1
3,my food stock is not the only one which is emp...,1
4,me ready to go at supermarket during the outbr...,-1


In [ ]:
test['labels'] = test['labels'].apply(target)
test.head()

,text,labels
0,trending new yorkers encounter empty supermark...,-1
1,when i couldnt find hand sanitizer at fred mey...,1
2,find out how you can protect yourself and love...,1
3,buying hits city as anxious shoppers stock up ...,-1
4,one week everyone buying baby milk powder the ...,0


In [ ]:
X = train['text']
y = train['labels']



In [ ]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 300
EMBEDDING_DIM = 100
tokenizer = text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 39349 unique tokens.


In [ ]:
x = tokenizer.texts_to_sequences(X.values)
x = sequence.pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x.shape)

X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.3,stratify=y)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


Shape of data tensor: (41119, 300)
(28783, 300) (28783,)
(12336, 300) (12336,)


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

Y_train=np.array(Y_train)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y_train)
print(Y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
Y_train=onehot_encoded

[0 1 1 ... 1 1 1]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
Y_test=np.array(Y_test)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y_test)
#print(Y_test)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
Y_test=onehot_encoded

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
405/405 [==============================] - 474s 1s/step - loss: 0.7427 - accuracy: 0.6648 - val_loss: 0.5454 - val_accuracy: 0.7926
Epoch 2/5
405/405 [==============================] - 467s 1s/step - loss: 0.3588 - accuracy: 0.8765 - val_loss: 0.4404 - val_accuracy: 0.8444
Epoch 3/5
405/405 [==============================] - 465s 1s/step - loss: 0.2206 - accuracy: 0.9307 - val_loss: 0.4696 - val_accuracy: 0.8440
Epoch 4/5
405/405 [==============================] - 462s 1s/step - loss: 0.1531 - accuracy: 0.9519 - val_loss: 0.5156 - val_accuracy: 0.8479
Epoch 5/5
405/405 [==============================] - 449s 1s/step - loss: 0.1177 - accuracy: 0.9638 - val_loss: 0.5518 - val_accuracy: 0.8413


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

386/386 [==============================] - 30s 77ms/step - loss: 0.5611 - accuracy: 0.8441
Test set
  Loss: 0.561
  Accuracy: 0.844
